# Projeto 2 - CDados

### Autores: Diego Guimarães Nuñez, Felipe Bakowski Nantes de Souza, Gustavo de Lima Mota

### Tema: Conseguimos prever o número de homicídios da cidade de São Paulo por ano?

### Features: PIB de São Paulo, Índice do Custo de Vida (ICV), Escolaridade, Tráfico de Drogas e Desemprego.

In [1]:
import basedosdados as bd
import pandas as pd

### Baixando as bases de dados utilizadas no projeto:

In [10]:
homicidio = bd.read_table(dataset_id='br_sp_gov_ssp',
table_id='ocorrencias_registradas',
billing_project_id="cdadosp2-404421")

Downloading: 100%|██████████| 153876/153876 [00:30<00:00, 5041.07rows/s]


In [11]:
homicidio

,ano,mes,id_municipio,regiao_ssp,homicidio_doloso,numero_de_vitimas_em_homicidio_doloso,homicidio_doloso_por_acidente_de_transito,numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,homicidio_culposo_por_acidente_de_transito,homicidio_culposo_outros,...,total_de_estupro,estupro,estupro_de_vulneravel,total_de_roubo_outros,roubo_outros,roubo_de_veiculo,roubo_a_banco,roubo_de_carga,furto_outros,furto_de_veiculo
0,2002,1,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,21,0
1,2002,2,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,32,0
2,2002,3,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,36,0
3,2002,4,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,1,<NA>,0,0,0,45,0
4,2002,5,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,1,<NA>,0,0,0,35,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153871,2014,8,3500204,São José do Rio Preto,0,0,0,0,0,0,...,1,<NA>,<NA>,0,<NA>,0,0,0,3,0
153872,2014,9,3500204,São José do Rio Preto,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,0,0
153873,2014,10,3500204,São José do Rio Preto,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,3,0
153874,2014,11,3500204,São José do Rio Preto,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,4,1


In [12]:
ivs = bd.read_table(dataset_id='br_ipea_avs',
table_id='municipio',
billing_project_id="cdadosp2-404421")

Downloading: 100%|██████████| 319681/319681 [06:22<00:00, 835.25rows/s] 


In [13]:
ivs # colunas que serão usadas para feature: "ivs", "ivs_infraestrutura_urbana", "ivs_capital_humano" e "ivs_renda_trabalho"
# índice de vulnerabilidade social.

,ano,sigla_uf,id_municipio,raca_cor,sexo,localizacao,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_trabalho,...,populacao_15_mais,populacao_15_17,populacao_15_24,populacao_16_18,populacao_18_mais,populacao_18_20,populacao_18_24,populacao_19_21,populacao_25_mais,populacao_65_mais
0,2000,AP,1600105,branco,total,total,0.572,0.775,0.573,0.369,...,606,80,225,76,526,56,145,58,381,44
1,2000,AP,1600279,branco,total,total,0.677,0.880,0.649,0.503,...,4765,507,1855,509,4258,761,1347,717,2911,129
2,2000,AP,1600303,branco,total,total,0.379,0.292,0.435,0.411,...,52086,5786,18687,5953,46301,5854,12902,5839,33399,2392
3,2000,AP,1600402,branco,total,total,0.656,0.777,0.622,0.569,...,889,105,335,120,784,120,229,104,555,55
4,2000,AP,1600535,branco,total,total,0.615,0.713,0.691,0.441,...,1387,101,452,117,1285,176,351,174,935,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319676,2010,MG,3104007,negro,mulher,rural,NaN,NaN,NaN,NaN,...,258,0,98,0,258,30,98,41,160,8
319677,2010,MG,3115508,negro,mulher,rural,NaN,NaN,NaN,NaN,...,77,0,14,0,77,0,14,0,63,0
319678,2010,MG,3133105,negro,mulher,rural,NaN,NaN,NaN,NaN,...,121,0,5,0,121,5,5,5,116,19
319679,2010,MG,3168705,negro,mulher,rural,NaN,NaN,NaN,NaN,...,38,0,0,0,38,0,0,0,38,0


## Refletindo sobre os problemas sociais presentes em São Paulo, um dos mais notáveis é a abundante taxa de criminalidade, especificando ainda mais, a taxa de homicídio. Visado a target (problema o qual é pretendido visualizar), foi-se pensado em possíveis targets quais estão ligadas com o tema tido em vista, que são: Desemprego, Nível de Escolaridade, Produto Interno Bruto (PIB), Custo de Vida (ICV) e Índice de Vulnerabilidade Social (ICV).
## Tendo em vista a target com suas respectivas features, é pretendido realizar a criação de um modelo com capacidade de previsão para a criminalidade, entendendo intrinsicamente as relações individuais de cada feature com a target. A partir da criação do modelo, é facilitada a visão das 'raízes' do problema, possibilitando possíveis tomadas de decisões pessoais e governamentais para tratar esse severo problema.

#### Primeiramente, vamos tratar a base de dados de homicidios e deixá-la mais visualizável

In [14]:
homicidio_colunas = homicidio.loc[:,('ano', 'mes', 'regiao_ssp', 'homicidio_doloso', 'homicidio_culposo_outros')] #selecionando colunas
homicidio['Homicidio_total'] = homicidio.loc[:, ['homicidio_doloso', 'homicidio_culposo_outros']].sum(axis=1) #homicidio total
homicidio_filtrado = homicidio.loc[:,('ano', 'mes', 'regiao_ssp', 'Homicidio_total')] #selecionando colunas

In [15]:
homicidio_filtrado

,ano,mes,regiao_ssp,Homicidio_total
0,2002,1,Presidente Prudente,0.0
1,2002,2,Presidente Prudente,0.0
2,2002,3,Presidente Prudente,0.0
3,2002,4,Presidente Prudente,0.0
4,2002,5,Presidente Prudente,0.0
...,...,...,...,...
153871,2014,8,São José do Rio Preto,0.0
153872,2014,9,São José do Rio Preto,0.0
153873,2014,10,São José do Rio Preto,0.0
153874,2014,11,São José do Rio Preto,0.0


 # Agora devemos tratar a base IVS, para que, consigamos utilizar as colunas:  IVS, IDHM (IDH por municipio), Propoção_vulnerabilidade_socioeconomica (proporção de pessoas com renda per capita inferior a meio salario mínimo e que gastam mais de uma hora até chegar no trabalho), proporção_sem_agua_esgoto, Índice de Gini

### Iremos analisar dos anos 2012 até 2014

In [21]:
desemprego_2012_x = desemprego_2012.transpose().reset_index()
for header,columns in desemprego_2012_x.items():
    if header == 'index':
        j = 0
        for i in columns:
            mes = i[:3]
            ano = i[-4:]
            if mes == 'jan':
                mes = 1
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'fev':
                mes = 2
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'mar':
                mes = 3
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'abr':
                mes = 4
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'mai':
                mes = 5
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'jun':
                mes = 6
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'jul':
                mes = 7
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'ago':
                mes = 8
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'set':
                mes = 9
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'out':
                mes = 10
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'nov':
                mes = 11
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'dez':
                mes = 12
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            j += 1

In [22]:
desemprego_2012_x

,index,%
0,1-2012,"8,0"
1,2-2012,"7,8"
2,3-2012,"7,7"
3,4-2012,"7,6"
4,5-2012,"7,5"
5,6-2012,"7,4"
6,7-2012,"7,1"
7,8-2012,"7,0"
8,9-2012,"6,8"
9,10-2012,"6,9"


In [23]:
anos = list(homicidio_2012.ano)

In [24]:
meses = list(homicidio_2012.mes)

In [25]:
homicidio_2012_x = homicidio_2012.reset_index()

In [26]:
for indice in range(len(anos)):
    homicidio_2012_x['index'].iloc[indice] = f'{meses[indice]}-{anos[indice]}'

In [27]:
homicidio_2012_x

,index,ano,mes,regiao_ssp,Homicidio_total
0,1-2013,2013,1,Presidente Prudente,0.0
1,2-2013,2013,2,Presidente Prudente,0.0
2,3-2013,2013,3,Presidente Prudente,0.0
3,4-2013,2013,4,Presidente Prudente,0.0
4,5-2013,2013,5,Presidente Prudente,0.0
...,...,...,...,...,...
23023,8-2014,2014,8,São José do Rio Preto,0.0
23024,9-2014,2014,9,São José do Rio Preto,0.0
23025,10-2014,2014,10,São José do Rio Preto,0.0
23026,11-2014,2014,11,São José do Rio Preto,0.0


In [28]:
#homicidio_2012_x = homicidio_2012_x[['index','regiao_ssp','Homicidio_total']].dropna()

In [29]:
homicidio_2012_x

,index,ano,mes,regiao_ssp,Homicidio_total
0,1-2013,2013,1,Presidente Prudente,0.0
1,2-2013,2013,2,Presidente Prudente,0.0
2,3-2013,2013,3,Presidente Prudente,0.0
3,4-2013,2013,4,Presidente Prudente,0.0
4,5-2013,2013,5,Presidente Prudente,0.0
...,...,...,...,...,...
23023,8-2014,2014,8,São José do Rio Preto,0.0
23024,9-2014,2014,9,São José do Rio Preto,0.0
23025,10-2014,2014,10,São José do Rio Preto,0.0
23026,11-2014,2014,11,São José do Rio Preto,0.0


In [30]:
df = desemprego_2012_x.join(homicidio_2012_x, how='inner', lsuffix='_datas')

In [31]:
df = df[['index_datas','%','regiao_ssp','Homicidio_total']].dropna()

## Df geral criado!

### Entretanto, ele tem apenas 1 feature, devemos preenchê-lo com as outras e garantir que os objetos sejam todos integers, ao trocar ',' por '.'

In [32]:
def substitui(df, coluna):
    for header,columns in df.items():
        j = 0
        if header == coluna:
            for i in columns:
                n = i.replace(',','.')
                df['%'].iloc[j] = n
                j += 1

In [33]:
substitui(df, coluna='%')

In [34]:
df #substituído !

,index_datas,%,regiao_ssp,Homicidio_total
0,1-2012,8.0,Presidente Prudente,0.0
1,2-2012,7.8,Presidente Prudente,0.0
2,3-2012,7.7,Presidente Prudente,0.0
3,4-2012,7.6,Presidente Prudente,0.0
4,5-2012,7.5,Presidente Prudente,0.0
5,6-2012,7.4,Presidente Prudente,0.0
6,7-2012,7.1,Presidente Prudente,0.0
7,8-2012,7.0,Presidente Prudente,0.0
8,9-2012,6.8,Presidente Prudente,0.0
9,10-2012,6.9,Presidente Prudente,0.0


 ## Agora iremos adicionar as outras features!

 ### Entretando, como elas estão em datas completamente distintas, teremos que refazer o df geral, de modo que, ocorra  a maior intersecção de datas possíveis

In [35]:
homicidio_filtrado

,ano,mes,regiao_ssp,Homicidio_total
0,2002,1,Presidente Prudente,0.0
1,2002,2,Presidente Prudente,0.0
2,2002,3,Presidente Prudente,0.0
3,2002,4,Presidente Prudente,0.0
4,2002,5,Presidente Prudente,0.0
...,...,...,...,...
153871,2014,8,São José do Rio Preto,0.0
153872,2014,9,São José do Rio Preto,0.0
153873,2014,10,São José do Rio Preto,0.0
153874,2014,11,São José do Rio Preto,0.0
